# Real Estate Scrape

In [1]:
import re
import time
import requests
import gmaps.datasets
import pandas as pd
import numpy as np
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
from config import username, password, api_key

### Login Automation

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

rand_short = np.random.uniform(0.000, 3.000, size = 5) 
rand_long = np.random.uniform(0.000, 9.000, size = 4) 

url = 'https://www.recolorado.com/' # visit real estate website
browser.visit(url)
time.sleep(rand_short[0])

browser.find_by_tag('a')[63].click() # login process, click 
time.sleep(rand_short[1])

browser.fill('loginid', username) # username 
time.sleep(rand_short[2])

browser.fill('password', password) # password 
time.sleep(rand_short[3])

browser.find_by_value('login').click() # click button
time.sleep(rand_long[0])
browser.windows[1].close_others() # close other windows



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/Users/brandon/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


In [3]:
time.sleep(rand_long[1] + rand_long[2])
mls_app = browser.find_by_xpath('//*[@id="appColumn99"]') # table_app.is_empty()
mls_app.click()
time.sleep(rand_short[4])
browser.windows[1].close_others() # close other windows

### Active Listings

In [4]:
# navigate from homepage to active listings
active_listings = browser.find_by_id('m_wm_w4_m_lv_ctrl0_m_lnkWatch')
active_listings.click()

In [5]:
Active_df_table = {'listing_id':[], 'city':[], 'address':[], 'property_type':[], 
                'price':[], 'total_square_ft':[], 'above_square_ft':[], 
                'bed':[], 'bath':[], 'list_date':[], 'hoa':[]}

time.sleep(3)
Active_soup = soup(browser.html, 'html.parser')
pages = Active_soup.find_all('span', class_='pagingLinks')[1].get_text()
pages = re.search('...(.*)N',pages)
pages = int(pages.group(1))

page_count = 10

In [6]:
for page in range(pages):
    Active_soup = soup(browser.html, 'html.parser')
    Active_table = Active_soup.find('table',
                                    class_='displayGrid nonresponsive ajax_display d26m_show clickableDisplay')
    Active_body = Active_table.tbody
    
    for row in Active_body: # gather data from each page
        Active_df_table['listing_id'].append(row.find_all('td', class_='d26m10')[0].get_text())
        Active_df_table['city'].append(row.find_all('td', class_='d26m12')[1].get_text())
        Active_df_table['address'].append(row.find_all('td', class_='d26m14')[0].get_text())
        Active_df_table['property_type'].append(row.find_all('td', class_='d26m12')[0].get_text())
        Active_df_table['price'].append(row.find_all('td', class_='d26m15')[0].get_text())
        Active_df_table['bed'].append(row.find_all('td', class_='d26m16')[0].get_text())
        Active_df_table['bath'].append(row.find_all('td', class_='d26m16')[1].get_text())
        Active_df_table['list_date'].append(row.find_all('td', class_='d26m5')[0].get_text())
        
        if row.td.span.a != 0 and page==0:
            Active_df_table['total_square_ft'].append(row.find_all('td', class_='d26m7')[2].get_text())
            Active_df_table['above_square_ft'].append(row.find_all('td', class_='d26m7')[1].get_text())
            Active_df_table['hoa'].append(row.find_all('td', class_='d26m7')[4].get_text())
            
        elif row.td.span.a != 0 and page>=0:
            Active_df_table['total_square_ft'].append(row.find_all('td', class_='d26m17')[1].get_text())
            Active_df_table['above_square_ft'].append(row.find_all('td', class_='d26m17')[0].get_text())
            Active_df_table['hoa'].append(row.find_all('td', class_='d26m17')[3].get_text())
    if (page+1)%7 != 0:
        
        if (page+1)%7 == 1 and pages < (page+7):
            page_count = page_count/2
        
        elif (page+1) == 8:
            page_count += 2
            
    else:
        
        if (page+1) >= (pages - 7) and page <= (pages - 6):
            page_count = page_count - 2

    browser.find_by_xpath(f'//*[@id="m_upPaging"]/span/a[{page_count}]').click()
    time.sleep(2)

browser.quit()       

In [7]:
Active_df = pd.DataFrame(Active_df_table)
# Sometimes the data is returned uneven, this is to test and correct for it
# print(len(Active_df_table['listing_id'])) # Active_df_table['listing_id'].pop(0)
Active_df.head()

,listing_id,city,address,property_type,price,total_square_ft,above_square_ft,bed,bath,list_date,hoa
0,8096560,Keenesburg,414 Quincy Rr Avenue,Residential,"$455,500","1,473","1,473",3,2,05/30/2022,Yes
1,7041478,Aurora,14363 E Gunnison Place,Residential,"$499,950","1,724","1,724",4,2,05/30/2022,Yes
2,9418476,Como,260 Nez Perce Road,Land,"$139,900",,,,,05/30/2022,Yes
3,2504111,Denver,"460 S MARION Parkway Unit# 406, Bldg C",Residential,"$639,900","1,272","1,272",2,2,05/30/2022,Yes
4,3205405,Severance,612 Sawyers Pond Drive,Residential,"$553,335","3,333","2,415",5,3,05/30/2022,Yes


### Google API, cross reference 

In [8]:
lat=[]
lng=[]
count=0

for house in Active_df.index:
    address = Active_df['address'][house].replace(' ','+')
    city = '+' + Active_df['city'][house].replace(' ','+')
    
    try:
        response = requests.get(
            f'https://maps.googleapis.com/maps/api/geocode/json?address={address},{city},+CO&key={api_key}')
        resp_json_payload = response.json()
        lat.append(resp_json_payload['results'][0]['geometry']['location']['lat'])
        lng.append(resp_json_payload['results'][0]['geometry']['location']['lng'])
        
    except:
        lat.append(np.nan)
        lng.append(np.nan)
        count +=1

print(f"{count} address's unavailable")

address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address unavaible
address un

In [9]:
Active_df.insert(1, "lng", lng, True)
Active_df.insert(1, "lat", lat, True)
Active_df.to_csv('Resources/active_houses.csv', index_label="House number")

Active_df.head()

,listing_id,lat,lng,city,address,property_type,price,total_square_ft,above_square_ft,bed,bath,list_date,hoa
0,8096560,40.112200,-104.510358,Keenesburg,414 Quincy Rr Avenue,Residential,"$455,500","1,473","1,473",3,2,05/30/2022,Yes
1,7041478,39.687664,-104.821043,Aurora,14363 E Gunnison Place,Residential,"$499,950","1,724","1,724",4,2,05/30/2022,Yes
2,9418476,39.246920,-105.734955,Como,260 Nez Perce Road,Land,"$139,900",,,,,05/30/2022,Yes
3,2504111,NaN,NaN,Denver,"460 S MARION Parkway Unit# 406, Bldg C",Residential,"$639,900","1,272","1,272",2,2,05/30/2022,Yes
4,3205405,40.524148,-104.851082,Severance,612 Sawyers Pond Drive,Residential,"$553,335","3,333","2,415",5,3,05/30/2022,Yes


### Map Vizualization

In [10]:
gmaps.configure(api_key=api_key)

In [11]:
Active_df = Active_df.dropna()

info_box_template = """
<dl>
<dt>Listing ID</dt><dd>{listing_id}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Price</dt><dd>{price}</dd>
<dt>Total Square Feet</dt><dd>{total_square_ft}</dd>
<dt>Bed & Bath</dt><dd>{bed}, {bath}</dd>
<dt>City</dt><dd>{city}</dd>
</dl>
"""
home_info = [info_box_template.format(**row) for index, row in Active_df.iterrows()]
locations = Active_df[["lat", "lng"]]

In [12]:
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=home_info)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))